In [ ]:
from hypergraph import Graph, node
from hypergraph.viz import visualize


@node(output_name="raw_documents")
def fetch_documents(query, search_term) -> list:
    return []


@node(output_name="raw_images")
def fetch_images(doc_ids, search_term: str, query) -> list:
    return []


@node(output_name="raw_metadata")
def fetch_metadata(api_key: str, doc_ids: list) -> dict:
    return {}


@node(output_name=("combined_data"))
def combine(raw_metadata, raw_images):
    return []


graph = Graph(
    nodes=[fetch_documents, fetch_images, fetch_metadata, combine],
    name="data_ingestion",
)

visualize(graph)

In [ ]:
from hypergraph import Graph, node
from hypergraph.nodes.gate import ifelse


# Two mutex branches, both produce "result"
@ifelse(when_true="fast_path", when_false="slow_path")
def choose_path(x: int) -> bool:
    return x >= 0


@node(output_name="result")
def fast_path(x: int) -> int:
    return x + 1


@node(output_name="result")
def slow_path(x: int) -> int:
    return x - 1


@node(output_name="summary")
def summarize(result: int) -> int:
    return result * 10


g = Graph([choose_path, fast_path, slow_path, summarize], name="mutex_outputs")

# Visualize (try both modes)
g.visualize(depth=0, separate_outputs=False)  # classic, primary

In [ ]:
g.visualize(depth=0, separate_outputs=True)

In [ ]:
from hypergraph import Graph, node

In [ ]:
@node(output_name="y")
def double(x: int) -> int:
    return x * 2


@node(output_name="z")
def square(y: int) -> int:
    return y**2


# ===============================
# RAG pipeline
# ===============================
@node(output_name="embedding")
def embed(text: str) -> list[float]:
    return [0.1] * 10


@node(output_name="docs")
def retrieve(embedding: list[float]) -> list[str]:
    return ["doc1", "doc2"]


@node(output_name="answer")
def generate(docs: list[str], query: str) -> str:
    return "Answer"


# ===============================
# Diamond pattern
# ===============================
@node(output_name="a")
def start(x: int) -> int:
    return x


@node(output_name="b")
def left(a: int) -> int:
    return a + 1


@node(output_name="c")
def right(a: int) -> int:
    return a * 2


@node(output_name="d")
def merge(b: int, c: int) -> int:
    return b + c


# ===============================
# Complex RAG (19 nodes)
# ===============================
@node(output_name="raw_text")
def load_data(filepath: str) -> str:
    return "raw content"


@node(output_name="cleaned_text")
def clean(raw_text: str) -> str:
    return raw_text.strip()


@node(output_name="tokens")
def tokenize(cleaned_text: str) -> list[str]:
    return cleaned_text.split()


@node(output_name="chunks")
def chunk(tokens: list[str], chunk_size: int) -> list[list[str]]:
    return [tokens[i : i + chunk_size] for i in range(0, len(tokens), chunk_size)]


@node(output_name="embeddings")
def embed_chunks(chunks: list[list[str]], model_name: str) -> list[list[float]]:
    return [[0.1] * 768 for _ in chunks]


@node(output_name="normalized_embeddings")
def normalize(embeddings: list[list[float]]) -> list[list[float]]:
    return embeddings


@node(output_name="index")
def build_index(normalized_embeddings: list[list[float]]) -> dict:
    return {"vectors": normalized_embeddings}


@node(output_name="query_text")
def parse_query(user_input: str) -> str:
    return user_input.strip()


@node(output_name="query_embedding")
def embed_query(query_text: str, model_name: str) -> list[float]:
    return [0.1] * 768


@node(output_name="expanded_queries")
def expand_query(query_text: str) -> list[str]:
    return [query_text, f"{query_text} synonym"]


@node(output_name="query_embeddings")
def embed_expanded(expanded_queries: list[str], model_name: str) -> list[list[float]]:
    return [[0.1] * 768 for _ in expanded_queries]


@node(output_name="candidates")
def search_index(index: dict, query_embedding: list[float], top_k: int) -> list[int]:
    return list(range(top_k))


@node(output_name="expanded_candidates")
def search_expanded(index: dict, query_embeddings: list[list[float]], top_k: int) -> list[int]:
    return list(range(top_k * 2))


@node(output_name="merged_candidates")
def merge_results(candidates: list[int], expanded_candidates: list[int]) -> list[int]:
    return list(set(candidates + expanded_candidates))


@node(output_name="retrieved_docs")
def fetch_documents(merged_candidates: list[int], chunks: list[list[str]]) -> list[str]:
    return [" ".join(chunks[i]) for i in merged_candidates if i < len(chunks)]


@node(output_name="context")
def format_context(retrieved_docs: list[str]) -> str:
    return "\n\n".join(retrieved_docs)


@node(output_name="prompt")
def build_prompt(context: str, query_text: str, system_prompt: str) -> str:
    return f"{system_prompt}\n\nContext:\n{context}\n\nQuery: {query_text}"


@node(output_name="raw_response")
def call_llm(prompt: str, temperature: float, max_tokens: int) -> str:
    return "Generated response..."


@node(output_name="final_answer")
def postprocess(raw_response: str) -> str:
    return raw_response.strip()

In [ ]:
complex_rag = Graph(
    nodes=[
        chunk,
        load_data,
        clean,
        tokenize,
        embed_chunks,
        normalize,
        build_index,
        parse_query,
        embed_query,
        expand_query,
        embed_expanded,
        search_index,
        search_expanded,
        merge_results,
        fetch_documents,
        format_context,
        build_prompt,
        call_llm,
        postprocess,
    ],
    name="rag_pipeline",
)
complex_rag.visualize()

In [ ]:
from hypergraph.viz import extract_debug_data

data = extract_debug_data(complex_rag, depth=0)

# Show unique heights
heights = set(n["height"] for n in data.nodes)
print(f"Unique heights: {sorted(heights)}")

# Show nodes with their heights
for n in sorted(data.nodes, key=lambda x: x["height"], reverse=True)[:10]:
    print(f"  {n['id']}: {n['height']}px")

In [ ]:
from hypergraph import ifelse


@node(output_name="cached_result")
def use_cache(query: str) -> str:
    """Return cached result."""
    return "cached answer"


@node(output_name="fresh_result")
def compute_fresh(query: str) -> str:
    """Compute fresh result."""
    return "fresh answer"


@ifelse(when_true="use_cache", when_false="compute_fresh")
def check_cache(query: str) -> bool:
    """Check if query is in cache."""
    return query in ["hello", "world"]


branching = Graph(nodes=[check_cache, use_cache, compute_fresh])
branching.visualize()

In [ ]:
data = extract_debug_data(branching, depth=0)
heights = set(n["height"] for n in data.nodes)
print(f"Unique heights: {sorted(heights)}")
for n in data.nodes:
    print(f"  {n['id']}: {n['height']}px, type={n.get('nodeType', '?')}")

In [ ]:
from hypergraph import route


@node(output_name="small_result")
def process_small(data: str) -> str:
    return "processed small"


@node(output_name="medium_result")
def process_medium() -> str:
    return "processed medium"


@node(output_name="large_result")
def process_large(data: str) -> str:
    return "processed large"


@route(targets=["process_small", "process_medium", "process_large"])
def classify_size(data: str) -> str:
    """Classify data by size."""
    if len(data) < 10:
        return "process_small"
    elif len(data) < 100:
        return "process_medium"
    return "process_large"


routing = Graph(nodes=[classify_size, process_small, process_medium, process_large])
routing.visualize()

## 9. Nested Graph (Hierarchical Composition)

Graphs can contain other graphs as nodes. Use `depth` to control expansion.

In [ ]:
# Inner graph: text processing pipeline
@node(output_name="cleaned")
def clean_text(text: str) -> str:
    return text.strip()


@node(output_name="normalized")
def normalize(cleaned: str) -> str:
    return cleaned.lower()


preprocess = Graph(nodes=[clean_text, normalize], name="preprocess")


# Outer graph using the inner graph
@node(output_name="result")
def analyze(normalized: str) -> dict:
    return {"length": len(normalized)}


workflow = Graph(nodes=[preprocess.as_node(), analyze])
print("depth=0 (collapsed):")
workflow.visualize(depth=2, separate_outputs=True)

In [ ]:
from hypergraph.viz import extract_debug_data

# Extract rendered debug data via Playwright
data = extract_debug_data(workflow, depth=1)

# Print report
data.print_report()

# Access edge issues programmatically
for edge in data.edge_issues:
    print(f"{edge.source} -> {edge.target}: {edge.issue}")
    print(f"  srcBottom={edge.src_bottom}, tgtTop={edge.tgt_top}")
    print(f"  vertDist={edge.vert_dist}, horizDist={edge.horiz_dist}")

# Access all data
print(f"Nodes: {len(data.nodes)}")
print(f"Edges: {len(data.edges)}")
print(f"Issues: {data.summary['edgeIssues']}")

In [ ]:
from hypergraph.viz import visualize

# In Jupyter notebook - shows with debug overlay tabs
visualize(workflow, depth=0, _debug_overlays=False, separate_outputs=True)

In [ ]:
# Level 1: Simple transform
@node(output_name="step1_out")
def step1(x: int) -> int:
    return x + 1


@node(output_name="step2_out")
def step2(step1_out: int) -> int:
    return step1_out * 2


inner = Graph(nodes=[step1, step2], name="inner")


# Level 2: Wrap inner + add validation
@node(output_name="validated")
def validate(step2_out: int) -> int:
    return step2_out


middle = Graph(nodes=[inner.as_node(), validate], name="middle")


# Level 3: Wrap middle + add logging
@node(output_name="logged")
def log_result(validated: int) -> int:
    print(f"Result: {validated}")
    return validated


outer = Graph(nodes=[middle.as_node(), log_result])

print("depth=0:")
outer.visualize(depth=0)

In [ ]:
print("depth=1:")
outer.visualize(depth=1)

In [ ]:
print("depth=2 (fully expanded):")
outer.visualize(depth=2)

In [ ]:
rag = Graph(nodes=[embed, retrieve, generate])
rag.visualize()

In [ ]:
diamond = Graph(nodes=[start, left, right, merge])
diamond.visualize()

## Simple 2-Node Graph

In [ ]:
@node(output_name="y")
def double(x: int) -> int:
    return x * 2


@node(output_name="z")
def square(y: int) -> int:
    return y**2


graph = Graph(nodes=[double, square])
graph.visualize()

## RAG Pipeline (3 nodes)

In [ ]:
@node(output_name="embedding")
def embed(text: str) -> list[float]:
    return [0.1] * 10


@node(output_name="docs")
def retrieve(embedding: list[float]) -> list[str]:
    return ["doc1", "doc2"]


@node(output_name="answer")
def generate(docs: list[str], query: str) -> str:
    return "Answer"


rag = Graph(nodes=[embed, retrieve, generate])
rag.visualize()

In [ ]:
@node(output_name="embedding")
def embed() -> list[float]:
    return [0.1] * 10


rag = Graph(nodes=[embed])
rag.visualize()

## With Type Hints

In [ ]:
rag.visualize(show_types=True)

## Wider Graph (Diamond Pattern)

In [ ]:
@node(output_name="a")
def start(x: int) -> int:
    return x


@node(output_name="b")
def left(a: int) -> int:
    return a + 1


@node(output_name="c")
def right(a: int) -> int:
    return a * 2


@node(output_name="d")
def merge(b: int, c: int) -> int:
    return b + c


diamond = Graph(nodes=[start, left, right, merge])
diamond.visualize()

## Light Theme

In [ ]:
diamond.visualize(theme="light")